# 从0到1训练大模型
# Part 2 MateConv Mini模型构建与预训练流程

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/80.png)

- **大模型训练流程概览**

从 **0到1** 训练大模型是一个复杂而系统的工程，需要涵盖从数据准备到模型部署的多个环节。以下是一个完整的流程框架：

| 流程            | 说明                                                                                           |
|:---------------:|--|
| **数据准备**        | 收集高质量、覆盖面广的训练数据，对其进行清洗、去噪和格式化处理。划分<br>训练集、验证集，并存储为高效读取的格式。这一步为模型提供了扎实的输入基础。 |
| **硬件与环境配置**  | 为模型训练准备高性能硬件（如 A800、A100 GPU），搭建分布式训练环境，<br>并优化深度学习框架的配置。这一步确保训练效率和稳定性。 |
| **分词器训练**      | 根据训练数据量和模型任务需求，选择适合的分词算法（如 BPE 或 <br>SentencePiece）。分词器决定了模型如何理解数据，是数据与模型的桥梁。 |
| **设计模型架构**    | 选择适合的模型结构（如 GPT、BERT），并配置参数量、层数、激活函数<br>等细节。对于大规模任务，可以结合领域特点定制模型。 |
| **预训练**        | 使用无监督任务从海量数据中提取通用知识，比如语言模型的自回归建模或<br>掩码建模。预训练的效果直接影响模型后续的微调能力。 |
| **意图对齐微调**        | 通过监督微调（SFT）或强化学习对齐（RLHF），让模型学习人类偏好，<br>避免输出无意义或有害内容。对齐步骤是模型实用化的关键。 |
| **特定优化微调**  | 在特定任务（如文本分类、问答）上微调模型，结合冻结与解冻层的策略进一步<br>优化性能，满足应用需求。                          |
| **模型量化**       | 通过剪枝、量化和知识蒸馏等技术优化模型，提高推理效率，降低计算与存储<br>成本，使模型更适合部署环境。                            |
| **部署与监控**      | 将模型部署到生产环境中，使用推理优化工具提升服务效率，同时通过实时<br>监控与用户反馈不断改进模型性能和可靠性。                     |

**在课程中我们呈现的环节有——**

| 流程            | MateConv Mini<br> (0.02B) | MateConv <br> (1B) | 单独的案例 | 在这节课之前里应该掌握 |
|:---------------:|:---------------------:|:-------------:|:-:|:-:|
| **数据准备**<br>（数据收集、数据清洗、数据预处理、<br>数据分词、数据编码、掩码构建……）| ✅ | ✅ | ✅ |✅大部分已完成<br>剩下的在<font color="red">**当前课程**|
| **硬件与环境配置**<br><br> （硬件配置、环境配置、代理设置）  | ✅                     | ✅             | ✅|✅ 已完成 |
| **分词器训练**      | ✅  | --              |--|✅ 已完成|
| **设计模型架构**    | ✅  | ✅     |✅| ✅大部分已完成<br>剩下的在<font color="red">**当前课程** |
| **预训练**<br>（预训练基础理论、配置文件构建、<br>各类优化方法、分布式配置、<br>模型监控与保存配置……）          | ✅                     | ✅             |--| <font color="red">**当前课程** |
| **意图对齐微调**        | ✅                     | ✅             |--|--|
| **特定优化微调**  | --                     | --             |✅|--|
| **模型量化**       | --                   | --             |✅|--|
| **部署与监控**     | ✅                     | --             |--|--|

在进行课程讲解时，每个环节我们将采用【技术概览 - MateConv Mini代码实现 - MateConv代码实现 - 深度原理或补充】的4段式结构进行讲解、你可以挑选自己最感兴趣的部分进行学习。

## 1 MateConv Mini的模型构建

MateConv Mini是基于LlaMA3.1 + DeepSeek构建的文字生成式模型，在课程中的《LlaMA架构原理精讲与复现》板块我们已详细解读过这个架构，核心结构如下 ↓ 

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/78.png" alt="描述文字" width="600">

**MateConv Mini** 的模型架构整体设计融合了 **LLaMA-like** 和 **DeepSeek-like** 的特点，该架构从 **输入嵌入层（Embeddings）** 开始，通过多层堆叠的 Transformer 模块完成特征建模，最终通过 Softmax 输出预测结果。其中、每个 Transformer 模块由 **多查询注意力（Grouped Multi-Query Attention with KV Cache）**、**Feed Forward 模块（或 MoE）** 以及归一化和残差连接组成。

- **1. Input & Embeddings**：将离散的输入序列（如单词、字符或 Token）转换为连续向量表示，提供初始语义信息。作为整个模型的输入，提供每个 Token 的嵌入表示，用于后续的注意力和前馈处理。

- **2. RMS Norm（RMS归一化）**：对输入数据进行均方根归一化，稳定模型的训练，并加速收敛。RMS Norm 相比 Layer Norm 更高效，因为它只需计算每个样本的均方根值，而无需计算均值。

- **3. Grouped Multi-Query Attention with KV Cache**：实现自注意力机制，通过计算 Query、Key 和 Value 的相关性来捕获序列的上下文依赖。然而，我们使用的是KV共享的注意力机制、同时在推理阶段缓存 Key 和 Value，避免重复计算历史上下文，这能显著提高生成效率并且降低模型参数量。除此之外，我们还在 Query 和 Key 上引入旋转位置编码，增强模型对位置关系的建模能力，特别适用于长序列。

- **4. Feed Forward（SwiGLU）与 MoE（Experts）**：对每个 Token 的表示单独进行非线性变换，增强模型的特征提取能力。其中FFN层使用门控激活单元，通过一组门控机制控制信息流动，能够捕获更复杂的特征，而MOE层则在多个专家模块中选择性激活一个或多个（通常互斥，即“Mutually Exclusive”），使模型在不同任务上具有更高的灵活性。

- **5. Residual Connection（残差连接）**：将输入直接添加到子模块的输出上，缓解梯度消失问题、使模型能够在学习额外特征时保留输入信息，从而更有效地学习深层次的表示。

- **6. Linear & Softmax**：线性层将 Transformer 模块的输出映射到词汇表大小，用于生成词分布。Softmax 将词分布转换为概率分布，完成预测。

结合之前在讲解LlaMA时构建的代码、我们将该架构的模型脚本放在了model.py文件中 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/76.png)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/77.png)

| **代码与构建思路差异**            | **本次预训练中使用的model.py脚本**                                                                                       | **LLaMA课程中构建的LLaMA.py脚本**                                                                                          |
|--------------------------|-------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------|
|**MOE架构**|实现了MOE模型、以及共享专家机制|实现了MOE模型、以及共享专家机制|
| **KV 缓存更新**          |直接针对推理模式下 `seqlen == 1`的情况进行了优化（拼接新 Token 到缓存）、适用于自回归中只输出一个token的情况（适用于大部分情况）。                                     |通用实现、在自回归流程中无论一次输出几个token都可以实现，按照首次调用（缓存为空）和非首次调用（缓存更新）的逻辑对缓存进行选择性更新。                                       |
| **掩码处理**            |使用固定掩码矩阵 `self.mask`，假设序列长度不会变化。                                            |动态扩展掩码矩阵，适配缓存增长时的长度需求。                                                      |
| **旋转位置编码**        |在扩展 KV 之前执行旋转位置编码、高效简单避免重复计算                                                                |在扩展 KV 之后执行旋转位置编码，适配不同长度的张量输入、但计算逻辑更为复杂                                                      |
| **注意力计算**          | - 支持 Flash Attention，未支持时手动加固定掩码矩阵计算注意力。                                      |优先使用 Flash Attention，支持动态扩展掩码以适配不同序列长度。                                      |
 **动态适配性**          |假设固定掩码与序列长度，扩展性有限。                                                            |动态掩码扩展和缓存机制，适配性更强，支持任意序列增长。                                             |
| **适配性场景**          |偏向推理中已知固定长度的场景、<font color="red">**整体效率更高**。                                                                  |适配更复杂的推理场景，支持动态长度序列处理、<font color="red">**拓展性和通用性更强**。                                                      |

同时，我们为这个模型构建了如下的参数设置 ↓ 该设置在LMConfig.py脚本中 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/79.png)

```python
from transformers import PretrainedConfig


class LMConfig(PretrainedConfig):
    model_type = "MateConv Mini"

    def __init__(
            self,
            dim: int = 512,  # 模型隐藏层维度
            n_layers: int = 8,  # Transformer 堆叠的层数
            n_heads: int = 16,  # 注意力头的数量
            n_kv_heads: int = 8,  # KV（Key 和 Value）共享头的数量
            vocab_size: int = 6400,  # 词汇表的大小
            hidden_dim: int = None,  # 前馈网络中隐藏层的维度，默认为 None 时将使用 dim 的倍数
            multiple_of: int = 64,  # 前馈网络中隐藏层维度需要是该值的整数倍
            norm_eps: float = 1e-5,  # LayerNorm 或 RMSNorm 的 epsilon 参数，用于数值稳定性
            max_seq_len: int = 512,  # 最大序列长度
            dropout: float = 0.0,  # Dropout 的比例
            flash_attn: bool = True,  # 是否使用 Flash Attention（更高效的注意力实现）
            ####################################################
            # 下面是关于 MOE（Mixture of Experts，专家网络）的特定配置
            # 当 use_moe 为 False 时，以下配置无效
            ####################################################
            use_moe: bool = False,  # 是否使用专家网络（MOE）
            num_experts_per_tok=2,  # 每个 Token 被分配的专家数量
            n_routed_experts=4,  # MOE 模型中的总专家数量
            n_shared_experts: bool = True,  # 是否启用共享专家（共享权重）
            scoring_func='softmax',  # 专家选择的评分函数，默认为 'softmax'
            aux_loss_alpha=0.01,  # 辅助损失的权重系数，用于保持专家负载平衡
            seq_aux=True,  # 是否在序列级别上计算辅助损失
            norm_topk_prob=True,  # 是否对 Top-K 专家选择的概率进行归一化
            **kwargs,
    ):
        # 模型的主要参数
        self.dim = dim
        self.n_layers = n_layers
        self.n_heads = n_heads
        self.n_kv_heads = n_kv_heads
        self.vocab_size = vocab_size
        self.hidden_dim = hidden_dim
        self.multiple_of = multiple_of
        self.norm_eps = norm_eps
        self.max_seq_len = max_seq_len
        self.dropout = dropout
        self.flash_attn = flash_attn
        ####################################################
        # MOE（专家网络）的相关配置
        ####################################################
        self.use_moe = use_moe  # 是否启用专家网络
        self.num_experts_per_tok = num_experts_per_tok  # 每个 Token 被分配的专家数量
        self.n_routed_experts = n_routed_experts  # 总的专家数量
        self.n_shared_experts = n_shared_experts  # 是否共享专家
        self.scoring_func = scoring_func  # 专家选择的评分函数
        self.aux_loss_alpha = aux_loss_alpha  # 辅助损失的权重系数
        self.seq_aux = seq_aux  # 是否在序列级别上计算辅助损失
        self.norm_topk_prob = norm_topk_prob  # 是否对 Top-K 专家选择概率进行归一化
        super().__init__(**kwargs)
```

- **模型参数量的计算**

在这些参数中、以下这些参数会直接影响模型的参数规模——

| **参数名称**        | **说明**                                                                                      |
|---------------------|-----------------------------------------------------------------------------------------------|
| **`dim`**           | 模型隐藏层的维度，影响各层的权重矩阵大小。                                                     |
| **`n_layers`**      | Transformer 堆叠的层数，每层的参数规模会累加。                                                 |
| **`n_heads`**       | 注意力头的数量，影响 Query、Key、Value 以及输出的维度分配。                                     |
| **`n_kv_heads`**    | KV（Key 和 Value）的共享头数，影响 Key 和 Value 的参数规模。                                     |
| **`vocab_size`**    | 词汇表大小，影响嵌入层和输出层的参数规模。                                                     |
| **`hidden_dim`**    | 前馈网络中隐藏层的维度，直接影响前馈层的参数规模。若未设置，则为 `dim * multiple_of`。          |
| **`use_moe`**       | 是否启用专家网络（MOE）。                                                                      |
| **`n_routed_experts`** | 专家网络中路由的总专家数量，影响 MOE 模块的参数规模。                                         |

基于这些关键参数、当前架构的总参数量可以分为以下几部分计算：

**1. 嵌入层参数量**

> - **公式**：`vocab_size * dim`
>   
> - 嵌入层的参数来自词汇表大小和嵌入维度。
>
> -    $$
   \text{Embedding Params} = 6400 \cdot 512 = 3,276,800
   $$

In [ ]:
X ==>  4 * nn.Linear  ==> Q\K\V  output

**2. 每层自注意力机制参数量**

> - 注意力机制的参数分为 Query、Key、Value 和输出权重（`wo`），但是由于我们使用了**kv共享**机制，且借用了参数`n_kv_heads`作为共享的基础、因此注意力机制的参数量实际计算如下——
> 1. **Query 权重矩阵**：
>    - Query 是全头独立计算，因此：
>      $$
     \text{Query Params} = dim \cdot dim = 512 \cdot 512 = 262,144
     $$
> 
> 2. **Key 权重矩阵**（共享头）：
>    - Key 的参数量基于 `n_kv_heads` 和切分后的维度：
>      $$
     \text{Key Params} = n\_kv\_heads \cdot head\_dim \cdot dim = 8 \cdot 32 \cdot 512 = 131,072
     $$
> 
> 3. **Value 权重矩阵**（共享头）：
>    - Value 的参数量与 Key 相同：
>      $$
     \text{Value Params} = 131,072
     $$
> 
> 4. **Output 权重矩阵**：
>    - 输出层需要将 `n_heads` 的多头输出合并为原始维度：
>      $$
     \text{Output Params} = dim \cdot dim = 512 \cdot 512 = 262,144
     $$
> 
> **合计注意力机制参数量**：
> - Attention Params = Q Params+K Params+V Params+O Param
> $$ 262,144 + 131,072 + 131,072 + 262,144 = 786,432 $$

**3.1 每层前馈网络参数量**

> - **公式**：
>  $$
  \text{FFN Params} = 2 \cdot (dim \cdot hidden\_dim) + hidden\_dim \cdot dim
  $$
> - `2 * (dim * hidden_dim)`：前馈网络中两层全连接权重。
> - `hidden_dim * dim`：前馈层输出投影权重。
> $$ \text{FFN Params} = 3 \cdot (dim \times hidden\_dim) = 3 \cdot (512 \times 2048) = 3,145,728 $$

**3.2 如果启用MOE网络架构**

> 在MateConv mini架构中、我们MOE的结构由专家、路由、以及共享专家三部分组成——
>
>1. **专家网络的参数量（Experts Params）**：每个专家是一个 `FeedForward` 层，总共有 `n_routed_experts` 个专家。
>
> **单个专家参数量**：来自 `FeedForward` 的参数：
>     $$
     \text{Single Expert Params} = 3 \cdot (dim \cdot hidden\_dim)
     $$
> - `w1`: $ dim \rightarrow hidden\_dim $
> - `w2`: $ hidden\_dim \rightarrow dim $
> - `w3`: $ dim \rightarrow hidden\_dim $
>
>  **所有专家的参数量**：
>   $$
   \text{Experts Params} = n\_routed\_experts \cdot \text{Single Expert Params}
   $$
>
> $$
   \text{Experts Params} = n\_routed\_experts \cdot 3 \cdot (dim \cdot hidden\_dim)
   $$
>   $$
   \text{Experts Params} = 4 \cdot 3 \cdot (512 \cdot 2048)
   $$
>   $$
   \text{Experts Params} = 4 \cdot 3 \cdot 1,048,576 = 12,582,912
   $$
>
> 2. **门控网络的参数量（Gating Params）**
>   - 在 `MoEGate` 中，门控网络使用一个权重矩阵 `self.weight`：
>     $$
     \text{Gating Params} = dim \cdot n\_routed\_experts
     $$
>
>    $$
   \text{Gating Params} = dim \cdot n\_routed\_experts
   $$
>   $$
   \text{Gating Params} = 512 \cdot 4 = 2,048
   $$
> 
> 3. **共享专家的参数量（Shared Expert Params，若启用）**
>   - 如果配置了共享专家（`n_shared_experts is not None`），还需要计算共享专家的参数。
>   - 共享专家是一个额外的 `FeedForward` 层，其参数量与单个专家相同：
>     $$
     \text{Shared Expert Params} = 3 \cdot (dim \cdot hidden\_dim)
     $$
>
>   $$
   \text{Shared Expert Params} = 3 \cdot (dim \cdot hidden\_dim)
   $$
>   $$
   \text{Shared Expert Params} = 3 \cdot (512 \cdot 2048)
   $$
>   $$
   \text{Shared Expert Params} = 3 \cdot 1,048,576 = 3,145,728
   $$
> - 结合以上部分，MOE 的总参数量为：
>   $$
   \text{MoE Params} = \text{Experts Params} + \text{Gating Params} + \text{Shared Expert Params}
   $$
>   $$
   \text{MoE Params} = 12,582,912 + 2,048 + 3,145,728
   $$
>   $$
   \text{MoE Params} = 15,730,688
   $$

4. **输出层参数量**

> - **公式**：
>  $$
  \text{Output Params} = vocab\_size \cdot dim
  $$
>  在本次架构中、我们使用了**Embedding 和 Output 层共享权重**的参数优化技巧，因此 Output 层就不需要单独计算参数量，只需计算一次 Embedding 参数量即可。

5. **每层总参数量**

> 每层参数量为自注意力机制和前馈网络的参数之和：
> $$
\text{Layer Params} = \text{Attention Params} + \text{FFN Params}
$$
> $$
\text{Layer Params} = 786,432 + 2,097,152 = 2,883,584
$$

6. **所有层参数量**

> Transformer 层数为 `n_layers = 8`：
> $$
\text{Total Layer Params} = n\_layers \cdot \text{Layer Params} = 8 \cdot 2,883,584 = 23,068,672
$$

7. **总参数量**

> 总参数量包含嵌入层和所有 Transformer 层参数：
> $$
\text{Total Params} = \text{Embedding Params} + \text{Total Layer Params}
$$
> $$
\text{Total Params} = 3,276,800 + 23,068,672 = 26,345,472
$$
> 即26.3MB（两千六百万参数）、即0.02B模型。

8. **如果启用MOE层**

> 如果启用MOE层、那每层参数量为自注意力机制和MOE参数之和：
> $$
\text{Layer Params} = \text{Attention Params} + \text{MOE Params}
$$
> $$
\text{Layer Params} = 786,432 + 15,730,688 = 16,517,120
$$
> 在总层数`n_layers = 8`的情况下，所有层总参数量为：
> $$\text{Total Layer Params} = n\_layers \cdot \text{Layer Params} = 8 \cdot 16,517,120 = 132,136,960
$$
> 整体架构的总参数量包含嵌入层和所有 Transformer 层参数：
> $$
\text{Total Params} = \text{Embedding Params} + \text{Total Layer Params}
$$
> $$
\text{Total Params} = 3,276,800 + 132,136,960 = 135,413,760
$$
> 即135.4MB（一亿三千五百万参数）、即0.1B模型。

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/78.png" alt="描述文字" width="600">

## 2 MateConv Mini的预训练代码全解

正如之前所说，这样的架构所需的算力是——

| 模型名称     | 模型参数量级      | 数据量级                | 硬件资源               | 训练时间              | 训练成本                |
| ------------ | ----------------- | ----------------------- | ---------------------- | --------------------- | ----------------------- |
| **MateConv mini** | 0.02B<br>（两千万参数） | 约60G（分词器约1G、预训练约35G、微调约20G） | RTX 3090 x2           | 全流程约3-4天          | AutoDL租赁，约250元     |
| **MateConv**      | 1B<br>（10亿参数）      | 约3~4T文本+1T代码（实际收集超过10T数据、如果你们有更大的运存可以使用更大的数据集） | A800 x 8，硬盘4T      | 全流程约1个多月（预训练约20天） | AutoDL租赁2.5\~3万元；购置设备需80\~100万元，电费约3000r |

预训练脚本 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/83.png)

运行全流程的代码 ↓

```shell
deepspeed --master_port 29500 --num_gpus=2 pretrain.py --epochs 15
```

开始训练 ↓

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/0d46acea891701fd573115782288e05.jpg" alt="0d46acea891701fd573115782288e05" style="zoom:50%;" />

训练结束 ↓ 训练流程大约18个小时。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/9f1b2321dc79550b1e0f194e165ff22.png" alt="9f1b2321dc79550b1e0f194e165ff22" style="zoom:50%;" />

### 2.1 深度学习训练的基本流程

深度学习模型的训练流程本质上是一个优化的流程，其**目标是通过不断迭代模型参数$w$，使模型在给定数据上的预测尽可能接近真实值**，简单来说，模型是“从错误中学习、通过损失函数的指导、不断调整自身的参数让预测更加准确”。为了实现这个目标、我们通常需要掌握优化算法、迭代流程等具体技术细节（<font color="red">**详细内容可参考《深度学习实战》课程**</font>）、但优化算法的内容本身繁多且难度较高、在这里我们跳开原始理论、以最快的速度、比较通俗的方式向大家展现深度学习的基本训练流程、为后续理解预训练的流程做好准备。

---

<font color="red">**在定义好模型之后、深度学习训练的基本步骤与流程如下**——

1. **数据分批/打乱/分训练集验证集等操作、定义使用的CPU/GPU等细节**

2. **初始化模型参数（Parameter Initialization）**

3. **训练循环（training loop）**：
> **正向传播（Forward Pass）**：输入数据通过模型，得到预测输出。
> 
> **计算损失（Compute Loss）**：根据模型的预测结果和真实标签，计算损失函数值。
> 
> **反向传播（Backward Pass）**：根据损失函数的值，计算模型中各参数的梯度（使用自动微分）。
> 
> **参数更新（Update Parameters）**：根据计算出的梯度，使用优化器更新模型的参数。
> 
> **梯度清零（Zero Gradients）**：为了避免梯度累积，需要在每次迭代前将梯度归零。

4. **训练监控与模型保存**

---

在实际训练时、我们可以自定义进行多少次循环、进行怎样的循环。同时，在深度学习训练过程中、你还需要掌握如下的概念——

- **Epoch**：是**整个训练数据集被完整地传递一次通过模型**的过程，模型见过一次完整的数据、叫做执行了一个epoch。
  
- **Batch 与 Batch Size**：在深度学习中，由于计算资源的限制，训练时不会一次性将整个数据集输入模型，而是将数据分成多个小组，每个小组称为一个 Batch。模型会逐个处理这些 Batch，并在每个 Batch 上完成一次正向传播和反向传播。Batch Size 是指 每个 Batch 中包含的样本数量。
  
- **梯度（Gradients）**：是优化过程中计算出的方向信息，用于指导权重更新。从数学上来说，梯度是**损失函数相对于模型参数的导数构成的向量**，梯度决定了参数更新的方向、指向损失函数下降最快的方向、而梯度值的大小影响参数更新的幅度。

- **优化算法（Optimizer）**：使用梯度来对参数进行迭代的实际算法、不同的优化算法有不同的迭代细节、但本质都是计算损失、反向传播求解梯度、进行参数更新的方式来迭代参数。常见的有Adam、SGD、RMSProp等等。

- **学习率（Learning rate）**：是控制**每次权重更新步幅大小**的超参数，通常记为$\eta$。学习率控制了优化步幅，影响训练稳定性和速度。

- **提前停止（Early_stopping）**：在训练过程中监控模型在验证集上的性能，当性能不再提升时，提前结束训练，以避免模型在训练集上过度拟合。

下面是执行深度学习训练时、最简单的一套伪代码——

```python
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset

# 假设我们有以下准备好的模块
# - 自定义的模型：MyModel
# - 数据集：MyDataset
# - 损失函数：loss_fn
# - 优化器：optimizer

# 定义超参数
num_epochs = 10        # 训练的轮数
batch_size = 32        # 批次大小
learning_rate = 0.001  # 学习率

# 数据准备
train_dataset = MyDataset(train=True)  # 定义训练数据集
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # 数据加载器

# 模型初始化
model = MyModel()  # 初始化模型
model = model.to("cuda" if torch.cuda.is_available() else "cpu")  # 将模型移至 GPU 或 CPU

# 损失函数和优化器
loss_fn = nn.CrossEntropyLoss()  # 假设是分类任务
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练循环
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    model.train()  # 进入训练模式

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # 将数据移动到设备
        inputs, targets = inputs.to("cuda"), targets.to("cuda")

        # 1. 正向传播
        outputs = model(inputs)

        # 2. 计算损失
        loss = loss_fn(outputs, targets)

        # 3. 梯度清零
        optimizer.zero_grad()

        # 4. 反向传播
        loss.backward()

        # 5. 更新参数
        optimizer.step()

        # 打印训练状态
        if batch_idx % 10 == 0:  # 每 10 个批次打印一次
            print(f"Batch {batch_idx}, Loss: {loss.item():.4f}")

    # (可选) 每个 epoch 完成后，验证模型性能
    print(f"Epoch {epoch + 1} Completed!")

# (可选) 保存模型
torch.save(model.state_dict(), "model.pth")
print("Training Completed!")

现在你知道了，深度学习的训练流程通常包括从数据加载开始，将输入数据经过模型的正向传播得到预测结果，然后通过损失函数计算预测值与真实值之间的差异，再通过反向传播算法计算模型参数的梯度，利用优化器更新模型参数，最后清零梯度以准备下一轮迭代。

**然而，在大模型训练中，这一流程被扩展为更复杂的分布式场景，涉及跨多机多卡的高效数据并行和模型并行**，使用更大的批次来稳定训练，同时引入梯度检查点、混合精度训练等优化技术以减少内存占用和计算开销。此外，大模型的训练还会包括预训练和微调两个阶段，分别用于学习通用特征和适应特定任务。通过这种分布式、高效的训练流程，大模型能够在大规模数据和复杂任务中表现出色。

### 2.2 分布式预训练的基础概念与代码配置

分布式（Distributed）是一种计算方式，指将计算任务分布到多个节点上进行协同处理，以提升计算效率或处理更大规模的任务。在深度学习中，**分布式训练的目标是利用多个设备（如 GPU 或 CPU）来加速模型的训练或支持训练更大的模型**。

在分布式中，你常常会听到这些基础概念——

- **进程（Process）**：**一个独立的程序运行实例、比如训练一个神经网络**。该实例本身拥有自己的内存空间、文件描述符等资源。**在分布式训练中，每个 GPU 通常只会被分配一个进程，每个进程独立处理模型计算任务**。进程之间相互独立，彼此的内存空间不能直接访问，需要通过进程间通信（如共享内存、消息队列）进行数据交换。

-  **节点（Node）**：在多机训练中，每台机器是一个节点、每个节点可能有多个 GPU，多个节点通过高速网络（如 InfiniBand）进行通信，完成分布式训练。每个 GPU 可以运行一个或多个进程（通常是一个）。

- **线程（Thread）**：线程是进程的执行单元，**通常对应程序的一条指令流（比如执行一段代码）**，一个进程可以包含多个线程，线程共享进程的内存和资源。线程共享进程的资源，而进程之间是独立的、线程创建和切换的开销比进程小。在深度学习中、多线程机制常常用于数据预处理、数据加载、并行模型计算等等。<font color="red">**大部分时候、想要实现多线程操作、我们只需调整线程相关的超参数即可、但是想要多进程操作、则必须使用PyTorch DDP以及DeepSpeed这样的分布式框架来进行辅助**。

-  **单机多卡（Single Node Multi-GPU）**：单机多卡是指在一台物理机器上使用多个 GPU 进行并行训练、也就是只有1个节点、但是可以有多个进程。在一台机器上、多个GPU之间通过共享内存或高速总线（如 PCIe）进行通信、含有PCIe功能的显卡价格会更昂贵。这种方式的 GPU 间通信延迟较低，数据传输通过共享内存完成。使用 `torch.nn.DataParallel` 或 `torch.nn.parallel.DistributedDataParallel` 可以方便实现单机多卡训练。

- **多机多卡（Multi-Node Multi-GPU）**：多机多卡是指使用多台物理机器，每台机器上有多个 GPU，通过高速网络将这些机器连接起来，共同进行分布式训练。节点之间的通信需要通过网络（如 Ethernet 或 InfiniBand），通信延迟比单机多卡高，还需要使用 NCCL、GLOO 或 MPI 等通信协议来通信后端。在需要利用数百甚至数千个 GPU 时，通常采用多机多卡训练方式。例如 GPT-3、BERT 等大规模模型的预训练。

不难发现，只有跨进程、跨设备的训练才需要用到多个设备，因此分布式训练是指跨进程的训练、多线程并行不是分布式运算的一部分。

在分布式训练中、我们有两种主要训练方式——

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/82.png)

#### 2.2.1 数据并行（Data Parallelism）
  
数据并行是将训练数据划分为多个不重叠的部分，并分配给不同的设备（如 GPU），每个设备上独立计算正向传播和反向传播，然后同步梯度更新参数。在数据并行中、**模型副本在每个设备上完全相同、而训练数据被分片**，每个设备只处理其中一部分、计算负载与设备数成正比。

在数据并行时，每个设备独立运行正向传播和反向传播、设备间同步梯度，更新模型参数。这种方式简单高效，适用于大多数场景、但是有设备之间存在通信开销。

- **为什么一定要通信？不通信不行吗？**

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/81.png)

上述图像展示了pytorch中的`DistributedDataParallel`类的具体流程——

1. **Dataloader（左侧）**：
   - 数据加载器（Dataloader）负责将训练数据划分为批次（Batch），然后分发给各个 GPU。
   - 每个 GPU 处理的数据来自一个分片，确保数据互不重叠。
<br><br>
2. **模型副本（紫色 LLM）**：
   - 每个 GPU 上都有一份相同的模型副本。
   - 所有设备共享同一组初始参数，但独立运行正向传播和反向传播。
<br><br>
3. **Forward/Backward Pass（正向和反向传播）**：
   - 每个 GPU 对其分配的数据进行正向传播（计算输出）和反向传播（计算梯度）。
   - 这些计算完全独立，因此可以并行进行。
<br><br>
4. **梯度同步（Synchronize Gradients）**：
   - 在反向传播完成后，各个 GPU 之间会通过通信操作（如 `all_reduce`）同步它们计算出的梯度。
   - 同步后的梯度是所有 GPU 的平均值，用于确保模型参数的一致性。
<br><br>
5. **模型更新（Update Model）**：
   - 每个 GPU 使用同步后的梯度更新自己的模型副本。
   - 更新后的模型在下一轮迭代时仍保持一致。

#### 2.2.2 模型并行（Model Parallelism）

模型并行是将模型的不同部分分配到不同的设备上，多个设备合作完成正向传播和反向传播。在模型并行中、数据在多个设备间共享、每个设备负责模型的一个子部分。常用于超大模型（如 GPT-3、PaLM 等）的训练。

在模型并行时、**数据还是会被分片、每次处理的数据是全局数据的一个子集**。同时，**分割在不同设备上的模型结构大多可以并行**，这样在一群可并行的设备处理完数据自己后、就会将信息传递给下一群可并行的设备。反向传播时，设备按相反顺序逐步计算梯度。这种方式能够训练内存需求极大的模型。但是实现起来极其复杂（因为需要对模型进行拆分），设备间通信更多。

在模型并行中，模型的不同部分（如网络层或参数分块）被分配到不同的设备，每个设备只负责处理一部分模型权重。因此，**模型的前向传播和反向传播需要跨设备传递中间计算结果和梯度**，前向传播时，每个设备必须将模型中间激活值（activation）传递给下一个设备。激活值的大小通常很大（取决于 Batch Size 和神经元数），传输这些值需要大量通信。反向传播时，设备需要将梯度值传递给计算前一层的设备。
每层梯度的大小与模型的权重规模相同，因此反向传播也会产生大量通信。为了解决这个问题，我们现在有管道并行（pipeline parallelism）、ZeRO优化、以及混合并行（Hybrid Parallelism）等更前沿的手段、不过这些手段在0.02B模型的训练流程中并不常见。

- **模型具体如何并行？**

**1.层级并行（Layer-wise Parallelism）**：

将模型的不同层分配到不同设备。例如，Transformer 的第 1-6 层分配到 GPU 0，第 7-12 层分配到 GPU 1。前向传播时，输入通过第 1-6 层后，将结果传递给第 7-12 层。反向传播时，梯度以相反顺序传递。

**例如**：
```python
import torch
import torch.nn as nn

class TransformerLayer(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.attention = nn.Linear(dim, dim)  # 简化的注意力层
        self.feedforward = nn.Linear(dim, dim)  # 简化的前馈层

    def forward(self, x):
        x = self.attention(x)
        x = self.feedforward(x)
        return x

# 分配层到不同设备
device_0 = torch.device("cuda:0")
device_1 = torch.device("cuda:1")

layers = [TransformerLayer(512).to(device_0) for _ in range(6)] + \
         [TransformerLayer(512).to(device_1) for _ in range(6)]

def forward(x):
    for i, layer in enumerate(layers):
        x = x.to(layer.attention.weight.device)  # 确保输入数据在对应设备上
        x = layer(x)
    return x

这种方式实现简单。适合模型深度较大、层数较多的情况。层之间依赖较强，设备间通信多。可能导致部分设备计算空闲（负载不均）。

**2. 算子并行（Operator Parallelism）**

将同一层中的算子（例如矩阵乘法、激活函数等）分配到不同设备。例如，Transformer 的注意力机制可以在多张 GPU 间分块计算。适合单层计算量巨大的情况、可均衡不同设备的负载。但是问题在于、通信开销大，因为算子之间的结果需要频繁交换、需要精细的实现和调度。

**3. 张量切分（Tensor Parallelism）**

将模型参数的张量切分到不同设备。例如，将 Transformer 中的注意力权重或前馈层的权重分为多个块，每个块分配给不同 GPU。前向传播时，各设备独立计算自己负责的部分，然后汇总。反向传播时，各设备计算梯度并同步。这种方式适合参数规模较大的情况、可避免单个设备内存不足的问题，但是实现非常复杂、需要频繁同步各种张量。

**例如**：

- 将每个 Transformer 层的权重矩阵沿某一维度切分到不同设备。例如，注意力权重分为两部分：
  - GPU 0：负责权重的左半部分。
  - GPU 1：负责权重的右半部分。
- 前向传播：
  - 每个设备独立计算其分配的部分，结果汇总后返回。
- 反向传播：
  - 梯度按张量分片同步。

```python
import torch

class ParallelAttention(nn.Module):
    def __init__(self, dim, device):
        super().__init__()
        self.q = nn.Linear(dim // 2, dim // 2).to(device)
        self.k = nn.Linear(dim // 2, dim // 2).to(device)

    def forward(self, x):
        q = self.q(x[:, :x.shape[1] // 2])  # 分片输入
        k = self.k(x[:, x.shape[1] // 2:])
        return q @ k.T

# GPU 分片
device_0 = torch.device("cuda:0")
device_1 = torch.device("cuda:1")

parallel_attention = ParallelAttention(512, device_0)
x = torch.randn(64, 512).to(device_0)
output = parallel_attention(x)
```

**4. 流水线并行（Pipeline Parallelism）**

将模型按层分配到不同设备，同时分割数据为多个小批次（Micro-Batches），以流水线方式并行处理。例如，GPU 0 处理第 1-6 层的第 1 小批次数据时，GPU 1 可以同时处理第 7-12 层的前一个小批次数据。这样提高设备利用率，但是同样实现非常复杂、通信开销高。

- 将 Transformer 的层按顺序划分到不同设备，并以小批次为单位进行流水线处理。
  - GPU 0：处理第 1-6 层。
  - GPU 1：处理第 7-12 层。
- 流水线机制：
  - 小批次 1：通过 GPU 0 后传递给 GPU 1。
  - 小批次 2：在 GPU 0 计算时，GPU 1 同时处理前一个小批次。

**例如**：
```python
# 使用 PyTorch 的 PipelineParallel 实现
from torch.distributed.pipeline.sync import Pipe
from torch.nn import Sequential

device_0 = torch.device("cuda:0")
device_1 = torch.device("cuda:1")

model = Sequential(
    *[TransformerLayer(512).to(device_0) for _ in range(6)],
    *[TransformerLayer(512).to(device_1) for _ in range(6)]
)
model = Pipe(model, chunks=2)  # 设置流水线并行的分块数
```

#### 2.2.3 完整train_torch函数解读

这个函数的作用是 **实现深度学习模型的单个 epoch 的训练过程**，涵盖了从数据加载、前向传播、损失计算、反向传播、学习率调整到模型保存的完整逻辑。它实现了对训练集的所有数据（分为多个批次）进行一轮完整训练。包括前向传播计算、梯度更新、学习率动态调整和模型保存等关键步骤。

同时还拓展了——
- **分布式支持**：如果使用分布式训练，仅在主节点上执行日志打印和模型保存操作。
- **混合精度训练**：使用 PyTorch 提供的混合精度工具（`torch.cuda.amp` 和 `GradScaler`）提升计算性能。
- **梯度累积**：通过累积多个小批次的梯度，以实现更大的等效批量大小，减少显存需求。
- **动态学习率调度**：根据当前迭代进度调整学习率（结合线性预热和余弦退火）。
- **模型状态保存**：定期保存模型和优化器的状态，以支持中断恢复和结果检查。
- **WandB 监控**：通过 WandB 平台记录训练指标，便于可视化和分析。

---

```python
def Logger(content):
    # 日志打印函数，只有在非分布式模式下或分布式主进程上会执行打印
    if not ddp or dist.get_rank() == 0:
        print(content)

def train_epoch(epoch, wandb):
    # 单个 epoch 的训练过程，包含数据加载、前向传播、反向传播、学习率调整和模型保存等逻辑
    start_time = time.time()  # 记录当前时间，用于计算每个 epoch 的耗时

    for step, (X, Y) in enumerate(train_loader):
        # 遍历训练数据加载器中的数据，每个 batch 包含输入 X 和目标 Y
        X = X.to(args.device)  # 将输入数据移动到指定设备（如 GPU）
        Y = Y.to(args.device)  # 将目标数据移动到指定设备

        # 根据当前迭代步数和总训练步数计算学习率
        lr = get_lr(epoch * iter_per_epoch + step, args.epochs * iter_per_epoch)

        # 更新优化器中所有参数组的学习率
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        with ctx:
            # 执行前向传播，计算模型输出和损失
            out = model(X, Y)
            # 将损失值归一化到梯度累积步数，以便于后续反向传播
            loss = out.last_loss / args.accumulation_steps

        # 使用混合精度训练缩放损失值后计算梯度
        scaler.scale(loss).backward()

        # 每完成指定的梯度累积步数后，执行优化器的参数更新步骤
        if (step + 1) % args.accumulation_steps == 0:
            scaler.unscale_(optimizer)  # 对梯度进行反缩放，以便执行梯度裁剪
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)  # 裁剪梯度以防止梯度爆炸

            scaler.step(optimizer)  # 更新模型参数
            scaler.update()  # 更新混合精度缩放因子

            optimizer.zero_grad(set_to_none=True)  # 将优化器中的梯度清零以准备下一次更新

        # 每隔 log_interval 步，记录和打印当前训练状态
        if step % args.log_interval == 0:
            spend_time = time.time() - start_time  # 计算当前耗时
            Logger(
                'Epoch:[{}/{}]({}/{}) loss:{:.3f} lr:{:.7f} epoch_Time:{}min:'.format(
                    epoch,  # 当前 epoch
                    args.epochs,  # 总共的 epoch 数
                    step,  # 当前 step
                    iter_per_epoch,  # 每个 epoch 的总 step 数
                    loss.item() * args.accumulation_steps,  # 实际的损失值
                    optimizer.param_groups[-1]['lr'],  # 当前学习率
                    spend_time / (step + 1) * iter_per_epoch // 60 - spend_time // 60  # 预计 epoch 剩余时间
                )
            )

            # 如果启用了 wandb 监控，则将相关指标记录到 wandb
            if (wandb is not None) and (not ddp or dist.get_rank() == 0):
                wandb.log({"loss": loss.item() * args.accumulation_steps,
                           "lr": optimizer.param_groups[-1]['lr'],
                           "epoch_Time": spend_time / (step + 1) * iter_per_epoch // 60 - spend_time // 60})

        # 每隔 save_interval 步保存模型和优化器的状态
        if (step + 1) % args.save_interval == 0 and (not ddp or dist.get_rank() == 0):
            model.eval()  # 切换模型到评估模式
            moe_path = '_moe' if lm_config.use_moe else ''  # 如果启用了 MOE，添加相关后缀
            ckp = f'{args.save_dir}/{args.model_name}.pth'  # 保存的模型路径（使用用户提供的模型名称）

            # 如果使用分布式训练，从分布式模型中提取实际的状态字典
            if isinstance(model, torch.nn.parallel.DistributedDataParallel):
                state_dict = model.module.state_dict()
            else:
                state_dict = model.state_dict()

            # 保存模型状态到文件
            torch.save(state_dict, ckp)
            Logger(f"保存模型到 {ckp}")

            # 保存优化器状态到文件
            optimizer_state_path = f'{args.save_dir}/{args.model_name}_optimizer.pth'
            torch.save(optimizer.state_dict(), optimizer_state_path)
            Logger(f"保存优化器状态到 {optimizer_state_path}")
            
            model.train()  # 切换回训练模式


### 2.3 学习率的余弦退火与线性预热机制

**学习率是深度学习模型训练中至关重要的超参数，它决定了每次参数更新的步长大小，对模型的收敛速度和最终性能有直接影响**。

如果学习率过大，优化过程可能会出现震荡甚至无法收敛，因为模型会跳过最优解；相反，如果学习率过小，模型的训练速度会变得非常缓慢，可能需要更多的迭代才能接近最优解，甚至会陷入局部最优。为了解决这些问题，动态调整学习率的方法（如学习率衰减、余弦退火、线性预热等）被广泛使用，这些策略可以在训练初期使用较大学习率加速收敛，在训练后期逐渐减小学习率以实现更精细的参数调整，最终提升模型性能和稳定性。因此，选择合适的学习率和调度策略是训练高效且稳定的深度学习模型的关键。

在本次训练中、我们一共使用了三种学习率调度机制、分别是Adam学习率自适应优化算法、以及全局学习率调节机制**余弦退火（Cosine Annealing）** 和 **线性预热（Linear Warmup）**，三种机制共同作用、用于动态调整深度学习训练过程中的学习率。

#### 2.3.1 线性预热（Linear Warmup）

在深度学习的训练初期，模型的权重还未适应数据分布，直接使用较大学习率可能导致不稳定的梯度更新，甚至损坏模型。为了让模型逐步适应学习任务，避免过大的学习率带来的不稳定性，**线性预热通过设置预热部署、让学习率从0开始逐步增长，在训练初期为模型提供一个稳定的优化过程**。具体来说，线性预热是让学习率在指定的预热步数（`warmup_iters`）内，从 $ 0 $ 平滑增长到目标学习率（`learning_rate`）。

公式为：
$$
\text{lr} = \text{learning\_rate} \times \frac{\text{it}}{\text{warmup\_iters}}
$$
- 当 $ \text{it} = 0 $：学习率为 $ 0 $。
- 当 $ \text{it} = \text{warmup\_iters} $：学习率为 $ \text{learning\_rate} $。
- 训练初期（$ \text{it} < \text{warmup\_iters} $），学习率随迭代次数线性增长。

---

我们可以用类似下面的代码来实现线性预热机制：
```python
if it < warmup_iters:
    return args.learning_rate * it / warmup_iters
```

- **`it`（Iteration）**：
  - 当前的迭代步数，表示已经执行了多少次参数更新。
  - 通常从 0 开始，在每次训练中的迭代次数逐渐增加。

- **`warmup_iters`（Warmup Iterations）**：
  - 学习率预热阶段的迭代步数，表示训练初期用于线性增长学习率的总步数。
  - 如果设置 `warmup_iters = 1000`，则在前 1000 次迭代中，学习率将从 0 增长到预设的目标学习率（`learning_rate`）。

- **`it < warmup_iters`**：
  - 判断当前迭代步数是否小于预热步数（`warmup_iters`）。如果是，则处于预热阶段。

---

- **什么时候可以尝试预热技巧？**

1. **梯度爆炸或数值不稳定**：
   - 随机初始化的参数可能导致梯度值过大，较大学习率会进一步放大这种不稳定性，最终使模型无法正确优化。
2. **难以找到优化方向**：
   - 初始阶段，模型的优化方向尚未确定，较大学习率可能使优化过程跳跃过潜在的收敛路径。
3. **损失震荡**：
   - 大学习率可能导致损失函数值剧烈波动，训练效率降低。

通过逐步增加学习率，模型可以以更平滑的方式进入稳定的优化状态。

---

- **什么时候可以不加预热？**

1. **使用自适应优化器**：
   - 如果采用 Adam、AdamW 等优化器，它们可以动态调整学习率，训练初期对大梯度有一定的容忍度。

2. **模型规模较小**：
   - 小型模型（例如简单的 CNN 或 MLP）在训练初期梯度变化较小，可以直接使用固定的学习率。

3. **初始化权重合理**：
   - 如果模型权重经过良好的初始化（如 LayerNorm 或 Xavier 初始化），梯度分布会更稳定，对学习率的依赖降低。

4. **训练资源有限**：
   - 对于需要快速收敛的场景（如快速原型开发），可以不加预热，直接使用固定学习率。

5. **学习率本身较小**：
   - 如果学习率设置得足够小，例如 $ 10^{-5} $，初期大梯度的影响会被大幅削弱，可以跳过预热。

#### 2.3.2 余弦退火机制（Cosine Annealing）

在训练的中后期，模型逐渐趋于收敛，学习率的作用需要减弱。**余弦退火**通过让学习率按照余弦函数的下降曲线变化，逐渐减小学习率，最终趋向于一个最小学习率值（`min_lr`）。与直接的线性下降不同，余弦退火曲线更加平滑，可以避免突降学习率带来的震荡。

在余弦退火阶段，学习率的计算公式为：
$$
\text{lr} = \text{min\_lr} + 0.5 \times (\text{max\_lr} - \text{min\_lr}) \times \left(1 + \cos\left(\pi \cdot \text{decay\_ratio}\right)\right)
$$
- $\text{max\_lr}$：初始学习率。
- $\text{min\_lr}$：学习率的最小值（训练末期）。
- $\text{decay\_ratio}$：当前迭代步数在衰减阶段的相对进度：
  $$
  \text{decay\_ratio} = \frac{\text{it} - \text{warmup\_iters}}{\text{lr\_decay\_iters} - \text{warmup\_iters}}
  $$

其中这个部分叫做**余弦系数（`coeff`）**：
  $$
  \text{coeff} = 0.5 \times (1 + \cos(\pi \cdot \text{decay\_ratio}))
  $$

- **理解 `decay_ratio`**

`decay_ratio` 是 **当前迭代步数在余弦退火阶段的相对进度**，它就是一个 **归一化的进度条**，用来表示余弦退火阶段的相对进度：
- 当 $ \text{decay\_ratio} = 0 $：表示退火刚开始，学习率在最高点。
- 当 $ \text{decay\_ratio} = 1 $：表示退火结束，学习率在最低点。
- 在整个退火阶段，`decay_ratio` 会线性从 0 增加到 1，并驱动余弦函数完成从 1 到 -1 的平滑过渡。

公式如下：
$$
\text{decay\_ratio} = \frac{\text{it} - \text{warmup\_iters}}{\text{lr\_decay\_iters} - \text{warmup\_iters}}
$$

其中——

1. **分子：$\text{it} - \text{warmup\_iters}$**
   - $ \text{it} $ 是当前迭代步数，表示当前训练到了哪一步。
   - $ \text{warmup\_iters} $ 是预热阶段的迭代步数，表示余弦退火阶段开始的迭代步。
   - $\text{it} - \text{warmup\_iters}$ 表示当前迭代步数相对于余弦退火开始时的步数的偏移量。
<br><br>
2. **分母：$\text{lr\_decay\_iters} - \text{warmup\_iters}$**
   - $ \text{lr\_decay\_iters} $ 是学习率退火结束时的迭代步数。
   - $ \text{warmup\_iters} $ 是学习率退火开始时的迭代步数。
   - $\text{lr\_decay\_iters} - \text{warmup\_iters}$ 表示余弦退火阶段的总步数。
<br><br>
3. **计算 `decay_ratio` 的逻辑**
   - 当 $\text{it} = \text{warmup\_iters}$ 时：
     $$
     \text{decay\_ratio} = \frac{0}{\text{lr\_decay\_iters} - \text{warmup\_iters}} = 0
     $$
     学习率刚进入余弦退火阶段，保持最大值。
<br><br>
   - 当 $\text{it} = \text{lr\_decay\_iters}$ 时：
     $$
     \text{decay\_ratio} = \frac{\text{lr\_decay\_iters} - \text{warmup\_iters}}{\text{lr\_decay\_iters} - \text{warmup\_iters}} = 1
     $$
     学习率退火到最小值。
<br><br>
   - 对于 $\text{warmup\_iters} < \text{it} < \text{lr\_decay\_iters}$：
     $\text{decay\_ratio}$ 在 $ [0, 1] $ 范围内平滑增长，表示当前迭代步数在退火阶段的相对进度。
<br><br>
- **结合公式中的 `decay_ratio`**
`decay_ratio` 被用于控制余弦函数中的角度进度：
$$
\cos\left(\pi \cdot \text{decay\_ratio}\right)
$$

- 当 $\text{decay\_ratio} = 0$ 时：
  $$
  \cos(0) = 1
  $$
  此时学习率处于最大值 $ \text{max\_lr} $。

- 当 $\text{decay\_ratio} = 1$ 时：
  $$
  \cos(\pi) = -1
  $$
  此时学习率处于最小值 $ \text{min\_lr} $。

- 在整个 $ \text{decay\_ratio} \in [0, 1] $ 的范围内：
  - $\cos$ 函数会从 1 平滑下降到 -1。
  - 最终计算学习率时，乘以 $ 0.5 \cdot (\text{max\_lr} - \text{min\_lr}) $，使学习率在最大值和最小值之间平滑过渡。

---

- **整体结合：线性预热 + 余弦退火 + Adam自适应调整**

1. **预热阶段**：
   - 学习率从 0 线性增长到初始学习率。
   - 阶段：$ \text{it} < \text{warmup\_iters} $。
   - 目的：让模型稳定训练，避免震荡。
<br><br>
2. **余弦退火阶段**：
   - 学习率从初始学习率按照余弦函数平滑下降到最小值。
   - 阶段：$ \text{warmup\_iters} \leq \text{it} \leq \text{lr\_decay\_iters} $。
   - 目的：让模型在中后期逐渐收敛，减少步长。
<br><br>
3. **固定学习率阶段**：
   - 当迭代步数超过衰减阶段，学习率固定为最小值。
   - 阶段：$ \text{it} > \text{lr\_decay\_iters} $。
   - 目的：防止学习率过低，停止训练。

结合线性预热和余弦退火的学习率曲线通常如下：
```
学习率
 ^
 |         /\
 |        /  \        ..........
 |       /    \      /
 |      /      \    /
 |_____/        \__/
 +-------------------------------------> 迭代次数
```
- 左侧为线性预热阶段。
- 中间为余弦退火阶段。
- 右侧为学习率固定阶段。

In [3]:
def get_lr(it, all):
    warmup_iters = args.warmup_iters
    lr_decay_iters = all
    min_lr = args.learning_rate / 10

    if it < warmup_iters:
        return args.learning_rate * it / warmup_iters
    if it > lr_decay_iters:
        return min_lr
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (args.learning_rate - min_lr)

代码说明 ↓

```python
def get_lr(it, all):
    # 获取当前学习率函数，输入参数为当前迭代步数 it 和总训练迭代步数 all

    warmup_iters = args.warmup_iters
    # 从参数中获取学习率预热的迭代步数（warmup_iters 表示预热阶段持续的迭代次数）

    lr_decay_iters = all
    # 设置学习率衰减的总迭代步数，通常等于训练的总迭代步数

    min_lr = args.learning_rate / 10
    # 设置最小学习率，通常是初始学习率的十分之一，表示学习率的下界

    if it < warmup_iters:
        # 如果当前迭代步数小于预热步数，则处于学习率预热阶段
        return args.learning_rate * it / warmup_iters
        # 学习率从 0 线性增长到初始学习率 args.learning_rate，按比例计算

    if it > lr_decay_iters:
        # 如果当前迭代步数超过衰减步数，说明已经进入训练末期
        return min_lr
        # 返回最小学习率，确保学习率不会进一步下降

    # 如果当前步数位于预热结束和衰减结束之间，则计算余弦退火学习率
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    # 计算衰减比例 decay_ratio，表示当前迭代步数在衰减阶段的相对进度
    # decay_ratio 的取值范围为 [0, 1]

    assert 0 <= decay_ratio <= 1
    # 断言：确保衰减比例在 [0, 1] 范围内，防止非法值导致错误

    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    # 使用余弦函数计算系数 coeff，该系数在 [1, 0] 之间变化
    # 初始时 coeff = 1（decay_ratio=0），逐渐衰减到 coeff = 0（decay_ratio=1）

    return min_lr + coeff * (args.learning_rate - min_lr)
    # 当前学习率等于最小学习率加上余弦系数与最大变化范围的乘积
    # 公式解释：学习率在最小学习率和初始学习率之间按照余弦函数平滑下降
```